In [23]:
import cx_Oracle
import os
import pandas as pd


connection = cx_Oracle.connect("pyora", "pyora", "ORCL19C")
sql = """select x.*,j.post_name, j.dep_name
  from (select t.emp_code,
               t.branch_id,
               t.report_name,
               t.log_date Log_Date,
               t.status_id,
               t.ip_address,
               t.host_name
          from MANA0809.TBL_REPORT_AUDIT_LOG@UATR_BACKUP2 t
         where to_date(t.log_date) = trunc(sysdate)) x,
       
       (select *
          from (select x.*,
                       row_number() over(partition by x.emp_code order by x.emp_code) as counts
                  from (select * from tableau_employ_dtl@uatr_backup2)
                       
                       x) y
         where y.counts = 1) j

 where j.emp_code = x.emp_code"""

df = pd.read_sql(sql,con=connection)


In [ ]:

data = pd.DataFrame(df)
cwd = os.getcwd()
path = r'C:\Users\356285\Desktop\AML\data\low1.csv'
df.to_csv(path)

data1 = pd.read_csv("low1.csv")
data2 = data1.drop_duplicates(subset=['EMP_CODE','LOG_DATE','IP_ADDRESS','REPORT_NAME','STATUS_ID','HOST_NAME','DEP_NAME'], keep='last')

data2['COUNT'] =1 
group_data = data2.groupby(['DEP_NAME','REPORT_NAME','POST_NAME'])['COUNT'].count() #sum function
group_data = group_data.reset_index()

group_data['dummy'] = 0
group_data3 = group_data.set_index(['DEP_NAME', 'REPORT_NAME', 'dummy'])
cwd = os.getcwd()

group_data3.to_excel('Low.xlsx', sheet_name='Sheet1')

dd = pd.read_excel('Low.xlsx', sheet_name='Sheet1')
dd.drop(axis=1, labels='dummy', inplace=True)


writer = pd.ExcelWriter('FinalData.xlsx', engine='xlsxwriter')
dd.to_excel(writer, sheet_name='Summary')
data2.to_excel(writer, sheet_name='Report',index=False)
workbook  = writer.book
worksheet = writer.sheets['Summary']
format1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})
start_row = 1
start_col = 1
end_row = len(dd)
end_cold = start_col

worksheet.conditional_format(start_row, start_col, end_row, end_cold,
                             {'type':     'cell',
                              'criteria': '>',
                              'value':    20,
                              'format':   format1})
writer.save()

